In [1]:
from keras.datasets import cifar10
import cv2 as cv2
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import model_from_json
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import os

Using TensorFlow backend.


In [0]:
def load_model(jsonfile,weightfiles):
    # load json and create model
    json_file = open(jsonfile, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weightfiles)
    print("Loaded model from disk")
    return loaded_model

In [0]:
def load_cifar10_data(img_rows, img_cols):
    (X_train,Y_train), (X_test, Y_test) = cifar10.load_data()
    
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
    X_test  = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_test[:,:,:,:]])
    
    classes = np.unique(Y_train)
    nClasses = len(classes)
    X_train.shape, X_test.shape
    Y_train  = to_categorical(Y_train,nClasses)
    Y_test  = to_categorical(Y_test,nClasses)    
    
    X_train = X_train.astype('float16')
    X_test = X_test.astype('float16')
    X_train = X_train / 255.
    X_test = X_test  / 255.
    
    return X_train, Y_train, X_test, Y_test, nClasses

In [4]:
X_train, Y_train, X_test, Y_test, nClasses = load_cifar10_data(100,100)
xdata = np.concatenate((X_train, X_test), axis=0)
ydata = np.concatenate((Y_train, Y_test), axis=0)
X_train,X_test,Y_train,Y_test = train_test_split(xdata, ydata, test_size=0.2, random_state=35,shuffle=True)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

170500096/170498071 [==============================] - 11s 0us/step


((48000, 100, 100, 3), (12000, 100, 100, 3), (48000, 10), (12000, 10))

In [0]:
jsonfile = '/content/drive/My Drive/python/modelConvNet573_100.json'
weightfiles = '/content/drive/My Drive/python/model_weightsConvNet573_100.h5'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
loaded_model = load_model(jsonfile,weightfiles)

Loaded model from disk


In [8]:
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 128)       204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 50, 128)       512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 128)      

In [0]:
Ypred = loaded_model.predict(X_test)
Ypred = np.argmax((Ypred),axis=1)
Ytest = np.argmax((Y_test),axis=1)
acc = accuracy_score(Ytest, Ypred)
prf = precision_recall_fscore_support(Ytest, Ypred,average='macro')

In [10]:
print('Precision = ', precision_score(Ytest, Ypred, average="macro"))
print('Recall = ', recall_score(Ytest, Ypred, average="macro"))  
print('F1 = ', f1_score(Ytest, Ypred, average="macro"))
print('Accuracy = ', accuracy_score(Ytest, Ypred))

Precision =  0.973498743350931
Recall =  0.9733442676350792
F1 =  0.973269388754441
Accuracy =  0.9735
